In [1]:
# !curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
# !unzip -q kagglecatsanddogs_3367a.zip
# !ls PetImages

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
import os
num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

Deleted 0 images


In [3]:
image_size = (32, 32)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 23410 files belonging to 2 classes.
Using 18728 files for training.
Found 23410 files belonging to 2 classes.
Using 4682 files for validation.


In [20]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.Rescaling(1.0 / 255),
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [21]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

In [22]:
def make_model(input_shape, num_classes, dual=False):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        
        if dual:
            x = layers.concatenate([layers.MaxPooling2D(3, strides=2, padding="same")(x),
                                    layers.AveragePooling2D(3, strides=2, padding="same")(x)])
            residual = layers.Conv2D(size*2, 1, strides=2, padding="same")(
            previous_block_activation
        )
        else:
            x = layers.MaxPooling2D(3, strides=2, padding="same")(x)
            residual = layers.Conv2D(size, 1, strides=2, padding="same")(
                previous_block_activation
        )
        
        
        # Project residual
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)




In [23]:
epochs = 50

In [24]:
model = make_model(input_shape=image_size + (3,), num_classes=2, dual=False)
# keras.utils.plot_model(model, show_shapes=False)

callbacks = [
#     keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss=keras.losses.BinaryCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

Epoch 1/50
586/586 [==============================] - 133s 228ms/step - loss: 0.6530 - accuracy: 0.6469 - val_loss: 0.6534 - val_accuracy: 0.6232
Epoch 2/50
586/586 [==============================] - 135s 231ms/step - loss: 0.5920 - accuracy: 0.7138 - val_loss: 0.6054 - val_accuracy: 0.7010
Epoch 3/50
586/586 [==============================] - 134s 228ms/step - loss: 0.5677 - accuracy: 0.7351 - val_loss: 0.5454 - val_accuracy: 0.7499
Epoch 4/50
586/586 [==============================] - 135s 231ms/step - loss: 0.5519 - accuracy: 0.7512 - val_loss: 0.6391 - val_accuracy: 0.6068
Epoch 5/50
586/586 [==============================] - 133s 228ms/step - loss: 0.5419 - accuracy: 0.7668 - val_loss: 0.8460 - val_accuracy: 0.6239
Epoch 6/50
586/586 [==============================] - 132s 226ms/step - loss: 0.5330 - accuracy: 0.7729 - val_loss: 0.6025 - val_accuracy: 0.6585
Epoch 7/50
586/586 [==============================] - 133s 227ms/step - loss: 0.5200 - accuracy: 0.7833 - val_loss: 0.5739 -

In [9]:
model = make_model(input_shape=image_size + (3,), num_classes=2, dual=False)
# keras.utils.plot_model(model, show_shapes=False)

callbacks = [
#     keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss=keras.losses.BinaryCrossentropy(label_smoothing=0.0),
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

Epoch 1/50
586/586 [==============================] - 129s 219ms/step - loss: 0.6454 - accuracy: 0.6417 - val_loss: 0.6907 - val_accuracy: 0.5145
Epoch 2/50
586/586 [==============================] - 128s 219ms/step - loss: 0.5586 - accuracy: 0.7190 - val_loss: 0.5673 - val_accuracy: 0.7345
Epoch 3/50
586/586 [==============================] - 128s 219ms/step - loss: 0.5260 - accuracy: 0.7413 - val_loss: 0.5641 - val_accuracy: 0.7085
Epoch 4/50
586/586 [==============================] - 128s 219ms/step - loss: 0.5021 - accuracy: 0.7582 - val_loss: 0.9805 - val_accuracy: 0.5831
Epoch 5/50
586/586 [==============================] - 128s 219ms/step - loss: 0.4918 - accuracy: 0.7690 - val_loss: 0.5094 - val_accuracy: 0.7721
Epoch 6/50
586/586 [==============================] - 128s 219ms/step - loss: 0.4794 - accuracy: 0.7758 - val_loss: 0.5581 - val_accuracy: 0.7670
Epoch 7/50
586/586 [==============================] - 128s 219ms/step - loss: 0.4639 - accuracy: 0.7847 - val_loss: 0.7415 -

In [10]:
model = make_model(input_shape=image_size + (3,), num_classes=2, dual=False)
# keras.utils.plot_model(model, show_shapes=False)

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

Epoch 1/50
586/586 [==============================] - 131s 223ms/step - loss: 0.6514 - accuracy: 0.6358 - val_loss: 0.6805 - val_accuracy: 0.5126
Epoch 2/50
586/586 [==============================] - 129s 220ms/step - loss: 0.5676 - accuracy: 0.7057 - val_loss: 0.7330 - val_accuracy: 0.5342
Epoch 3/50
586/586 [==============================] - 129s 220ms/step - loss: 0.5349 - accuracy: 0.7348 - val_loss: 0.5538 - val_accuracy: 0.7362
Epoch 4/50
586/586 [==============================] - 129s 220ms/step - loss: 0.5143 - accuracy: 0.7538 - val_loss: 0.6235 - val_accuracy: 0.6871
Epoch 5/50
586/586 [==============================] - 129s 220ms/step - loss: 0.4912 - accuracy: 0.7664 - val_loss: 0.4917 - val_accuracy: 0.7497
Epoch 6/50
586/586 [==============================] - 129s 220ms/step - loss: 0.4771 - accuracy: 0.7781 - val_loss: 0.5145 - val_accuracy: 0.7572
Epoch 7/50
586/586 [==============================] - 129s 219ms/step - loss: 0.4615 - accuracy: 0.7857 - val_loss: 0.4723 -

In [11]:
model = make_model(input_shape=image_size + (3,), num_classes=2, dual=False)
# keras.utils.plot_model(model, show_shapes=False)

callbacks = [
    keras.callbacks.ModelCheckpoint("ls01_save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss=keras.losses.BinaryCrossentropy(label_smoothing=0.01),
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

Epoch 1/50
586/586 [==============================] - 130s 221ms/step - loss: 0.6448 - accuracy: 0.6424 - val_loss: 0.6407 - val_accuracy: 0.6726
Epoch 2/50
586/586 [==============================] - 129s 221ms/step - loss: 0.5741 - accuracy: 0.7069 - val_loss: 0.6192 - val_accuracy: 0.6469
Epoch 3/50
586/586 [==============================] - 129s 221ms/step - loss: 0.5369 - accuracy: 0.7392 - val_loss: 0.5505 - val_accuracy: 0.7078
Epoch 4/50
586/586 [==============================] - 129s 221ms/step - loss: 0.5215 - accuracy: 0.7488 - val_loss: 0.5363 - val_accuracy: 0.7364
Epoch 5/50
586/586 [==============================] - 129s 221ms/step - loss: 0.5054 - accuracy: 0.7607 - val_loss: 0.5483 - val_accuracy: 0.7061
Epoch 6/50
586/586 [==============================] - 129s 220ms/step - loss: 0.4894 - accuracy: 0.7728 - val_loss: 0.5250 - val_accuracy: 0.7488
Epoch 7/50
586/586 [==============================] - 131s 224ms/step - loss: 0.4753 - accuracy: 0.7811 - val_loss: 0.5118 -